## Siniestro Viales

Importando librerias 

In [1]:
import numpy as np 
import pandas as pd 
#import pandas_alive
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import geojson
import folium
from folium.plugins import HeatMap
from summarytools import dfSummary


 ##ETL Extracción, transformación y carga

In [2]:
homicidios_hechos = pd.read_excel("homicidios.xlsx", sheet_name="HECHOS")
homicidios_victimas = pd.read_excel("lesiones.xlsx", sheet_name="VICTIMAS")
comunas= pd.read_excel("comunas.xlsx")

Tratamiento de los datos del dataset homicidios - hechos

In [3]:
homicidios_hechos

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,NaN,MOM,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,NaN,TERRADA,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,4071.0,NaN,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS


In [4]:
homicidios_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Examen de nulos y duplicados

In [5]:
homicidios_hechos.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

Se analizan las columnas con datos nulos

In [6]:
homicidios_hechos[["Altura", "Calle", "Dirección Normalizada", "Cruce"]][:5]

,Altura,Calle,Dirección Normalizada,Cruce
0,NaN,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...","FERNANDEZ DE LA CRUZ, F., GRAL. AV."
1,NaN,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",DE LOS CORRALES AV.
2,2034.0,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,NaN
3,NaN,LARRAZABAL AV.,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.","VILLEGAS, CONRADO, GRAL."
4,NaN,SAN JUAN AV.,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.","SAENZ PE?A, LUIS, PRES."


In [7]:
#filtramos las filas donde la columna tiene valores nulos y seleccionamos las columnas especificadas para esas filas
homicidios_hechos[homicidios_hechos["Dirección Normalizada"].isnull()][["Dirección Normalizada", "Cruce","Altura", "Calle"]]

,Dirección Normalizada,Cruce,Altura,Calle
38,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
106,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA
119,NaN,NaN,NaN,NaN
180,NaN,NaN,NaN,AUTOPISTA PERITO MORENO
181,NaN,NaN,NaN,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.
313,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
546,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
621,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA


Tras examinar datos nulos, que representan el 5,1% del dataset; distribuidos en tres columnas Altura, Dirección Normalizada, Cruce y Calle.

In [8]:
# Se elimina la columna "Altura"
homicidios_hechos= homicidios_hechos.drop("Altura", axis=1)
homicidios_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

In [9]:
homicidios_hechos.duplicated().sum()

0

Transformación de los datos
Se regularizan los tipos de datos de las columnas

In [10]:
homicidios_hechos["HH"].unique()

array([4, 1, 7, 0, 5, 18, 19, 15, 11, 22, 16, 9, 23, 6, 10, 17, 12, 8, 20,
       21, 14, 3, 2, 13, 'SD'], dtype=object)

In [11]:
# Cambio el dato horas  a tipo de dato numerico y luego a entero 
homicidios_hechos["HH"] = pd.to_numeric(homicidios_hechos ["HH"], errors="coerce")
homicidios_hechos["HH"] = homicidios_hechos["HH"].astype("Int64")

In [12]:
# Filtrar filas con posiciones no válidas
valid_positions = ~(homicidios_hechos['pos x'].str.contains('[^0-9.,-]') | homicidios_hechos['pos y'].str.contains('[^0-9.,-]') | (homicidios_hechos['pos x'] == '.') | (homicidios_hechos['pos y'] == '.'))

# Establecer posiciones no válidas en NaN
homicidios_hechos.loc[~valid_positions, ['pos x', 'pos y']] = np.nan

# Convertir las cadenas a tipo numérico (float)
homicidios_hechos['pos x'] = homicidios_hechos['pos x'].str.replace(',', '.').astype(float)
homicidios_hechos['pos y'] = homicidios_hechos['pos y'].str.replace(',', '.').astype(float)

In [13]:
homicidios_hechos.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS


La columna XY (CABA) posee coordenadas para poder ubicar en un mapa , las extraemos para poder utiliza

In [14]:
import re

In [17]:
def extraer_coordenadas(texto):
    # Utilizamos una expresión regular para extraer las coordenadas
    coordenadas = re.findall(r'\d+\.\d+', texto)
    if len(coordenadas) == 2:
        return float(coordenadas[0]), float(coordenadas[1])
    else:
        return None, None

# Aplicar la función a la columna 'XY (CABA)'
homicidios_hechos['coordenada_x'], homicidios_hechos['coordenada_y'] = zip(*homicidios_hechos['XY (CABA)'].map(extraer_coordenadas))

# Filtrar los registros que no tienen coordenadas válidas
homicidios_hechos = homicidios_hechos[(homicidios_hechos['coordenada_x'].notnull()) & (homicidios_hechos['coordenada_y'].notnull())]
homicidios_hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,coordenada_x,coordenada_y
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878


In [18]:
homicidios_hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,coordenada_x,coordenada_y
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878


In [19]:
#Se observan las columnas resultantes
homicidios_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO', 'coordenada_x', 'coordenada_y'],
      dtype='object')

Para agregar valor al dataset, agregaremos la descripción de cada una de las comunas, con los Barrios que la conforman.

Para eso utilizaremos el dataset 'comunas' importado junto al dataset homicidios. Este dataset fue descargado del siguiente sitio web: https://buenosaires.gob.ar/comunas

In [20]:
comunas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         15 non-null     int64  
 1   OBJETO     15 non-null     object 
 2   COMUNAS    15 non-null     int64  
 3   BARRIOS    15 non-null     object 
 4   PERIMETRO  15 non-null     float64
 5   AREA       15 non-null     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 852.0+ bytes


In [21]:
comunas = comunas.rename(columns={'COMUNAS':'Comuna'})

In [22]:
print(homicidios_hechos.columns)

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO', 'coordenada_x', 'coordenada_y'],
      dtype='object')


In [23]:
comunas.columns

Index(['ID', 'OBJETO', 'Comuna', 'BARRIOS', 'PERIMETRO', 'AREA'], dtype='object')

In [24]:
print("Columnas en homicidios_hechos:", homicidios_hechos.columns)
print("Columnas en comunas:", comunas.columns)


Columnas en homicidios_hechos: Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO', 'coordenada_x', 'coordenada_y'],
      dtype='object')
Columnas en comunas: Index(['ID', 'OBJETO', 'Comuna', 'BARRIOS', 'PERIMETRO', 'AREA'], dtype='object')


In [25]:
#Cambio la primer letra a mayúscula
homicidios_hechos.columns = [x.capitalize() for x in homicidios_hechos.columns]
# Reemplazo guiones por espacios
homicidios_hechos.columns = homicidios_hechos.columns.str.replace("_", " ")
#Renombra columnas 
homicidios_hechos= homicidios_hechos.rename(columns={"N victimas": "Num víctimas",
                                            "Aaaa": "Año",
                                            "Mm":"Mes",
                                            "Dd": "Día",
                                            "Hora": "Hora completa",
                                            "Hh": "Hora"}) 


In [24]:
print(homicidios_hechos.columns)

Index(['Id', 'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora completa',
       'Hora', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Coordenada x', 'Coordenada y'],
      dtype='object')


In [26]:
# Imprime los valores únicos en la columna 'COMUNA' en ambos DataFrames
print(homicidios_hechos['Comuna'].unique())
print(comunas['Comuna'].unique())


[ 8  9  1 11 15  4  7 12  3 13 14 10  6  2  5]
[ 2  5  6  7  9 10 11 12 14  3 15  8  4  1 13]


In [27]:
#Se realiza el merge a través de la columna en comun `Comuna` y nos va a quedar anexada la columna `Barrios`

homicidios_hechos=homicidios_hechos.merge(comunas, on='Comuna', how="left")

#Modifico las mayúsculas por minúsculas
homicidios_hechos=homicidios_hechos.rename(columns={
    "BARRIOS": "Barrios"
})

In [28]:
homicidios_hechos.head(3)

,Id,Num víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Lugar del hecho,Tipo de calle,...,Participantes,Victima,Acusado,Coordenada x,Coordenada y,ID,OBJETO,Barrios,PERIMETRO,AREA
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378,12,LIMITE COMUNAL,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI,20935.012513,2.221690e+07
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420,5,LIMITE COMUNAL,LINIERS - MATADEROS - PARQUE AVELLANEDA,21411.738344,1.650531e+07
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878,14,LIMITE COMUNAL,CONSTITUCION - MONTSERRAT - PUERTO MADERO - RE...,36102.201573,1.779457e+07


Las colummas tienen nombres en mayúsculas y también en minúsculas. Se estandarizan a modo de buena práctica

Al analizar los datos de la columna Día se muestra solo el numero del día y no el nombre correspondiente por lo que se crea la columna Día semana donde se corresponde cada fecha con un dia de la semana para poder analizar luego como se relacionan los siniestros con los dias de la semana

In [29]:
#Consulto la columna Día
homicidios_hechos.Día

0       1
1       2
2       3
3      10
4      21
       ..
677    13
678    20
679    30
680    15
681    18
Name: Día, Length: 682, dtype: int64

In [30]:
#Creo una columna nueva `Día semana` que contenga el nombre del día de la semana
homicidios_hechos['Día semana'] = homicidios_hechos['Fecha'].dt.strftime('%A')

#Creo un diccionario para modificar a español los nombres de los dás de la semana y lo aplico a la columna nueva
dic={  'Friday':'Viernes',
       'Saturday': 'Sabado',
       'Sunday': 'Domingo',  
       'Thursday': 'Jueves',
        'Monday': 'Lunes',
        'Wednesday': 'Miercoles',
       'Tuesday': 'Martes'}
homicidios_hechos['Día semana'] =homicidios_hechos['Día semana'].replace(dic)

In [31]:
homicidios_hechos.head(1)

,Id,Num víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Lugar del hecho,Tipo de calle,...,Victima,Acusado,Coordenada x,Coordenada y,ID,OBJETO,Barrios,PERIMETRO,AREA,Día semana
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,MOTO,AUTO,98896.782384,93532.434378,12,LIMITE COMUNAL,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI,20935.012513,2.221690e+07,Viernes


La colmuna Comuna se modifica agregando el termino Comuna antes del número, a fin de lograr una variable categórica que me simplifique el análisis

In [32]:
homicidios_hechos["Comuna"].unique()

array([ 8,  9,  1, 11, 15,  4,  7, 12,  3, 13, 14, 10,  6,  2,  5],
      dtype=int64)

In [33]:
#Agrego la palabra Comuna delante del dato numérico
#se aplica la función lambda a cada fila del dataframe 
homicidios_hechos["Comuna"] = homicidios_hechos.apply(lambda x: "Comuna "+str(x["Comuna"]), axis=1)

In [34]:
homicidios_hechos["Comuna"].unique()

array(['Comuna 8', 'Comuna 9', 'Comuna 1', 'Comuna 11', 'Comuna 15',
       'Comuna 4', 'Comuna 7', 'Comuna 12', 'Comuna 3', 'Comuna 13',
       'Comuna 14', 'Comuna 10', 'Comuna 6', 'Comuna 2', 'Comuna 5'],
      dtype=object)

Se realiza un resumen descriptivo de los datos en homicidios-hechos

In [35]:
dfSummary(homicidios_hechos)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Id[object],1. 2016-00012. 2019-00513. 2019-00434. 2019-00445. 2019-00456. 2019-00467. 2019-00478. 2019-00489. 2019-004910. 2019-005011. other,1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)672 (98.5%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADXElEQVR4nO3cwWkbYRRG0TdGOPZGQhhcQtKDi0ixKUJFpAdjPIvY8WaySBYpINwB5ZwK/sWFh2A+Ldu2DRRu9n4A/w+xkREbGbGRERsZsZERGxmxkREbGbGRuVmW5cuyLHd7P4TrdzMzn2fm094P4fo5o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRG5mZmvs/Mz70fwvVb/IsRFWeUjNjIWFeRsa4i44ySERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZExeCFj8ELGGSUjNjIGL2QMXsg4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjNjIWFeRsa4i44ySERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZGxriJjXUXGGSUjNjIGL2QMXsg4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZGxriJjXUXGGSUjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiMjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyy8zcz8zt3g/hqn1s2/Z2eHh4+Ho8Hs97v4brta7ry7Is3w7H4/H89PT0djqd3vd+FNfn9fX17nK5nJ+fn28PMzOn0+n98fHxx94P42rdz/iBQEhsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERuYw8/szkL0fwnX6u63Duq4vl8vlPH8+A4F/bV3Xl5n58Fk4hY9t296sq8j4gUBGbGTERkZsZMRGRmxkxEZGbGTERuYXwgdpOf758ysAAAAASUVORK5CYII="">",0(0.0%)
2,Num víctimas[int64],Mean (sd) : 1.0 (0.2)min < med < max:1.0 < 1.0 < 3.0IQR (CV) : 0.0 (5.8),3 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABs0lEQVR4nO3XQWrCUBCH8ZkiqQ/khZCaY3gAD+FhPYRbTxLkZdGoVF53hVJaTbToX77fNplhFh8RPedswKN7ufcBwCUm515w92Bmxcj9x5xzP3IW+PJnqO4e6rpexRirMcu7rtu5+5pYca1zX9Qixlgtl8u+LMv9kMUppelms6nati3MjFBxlbM//WZmZVnum6Z5H7E/jJgBfuDPFCQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiRM7n3AM3L3YGbFyPFjzrm/5T3PgFBvzN1DXderGGM1Zr7rup27r4n1u4tCTSlNhy5OKU1Pp9PEzKK7Dz5MWJzNZvPFYvERQjgMGez7/nW73c7btn1z9+6/DnxQh5zz/reHnwblSFGwSZ17AAAAAElFTkSuQmCC"">",0(0.0%)
3,Fecha[datetime64[ns]],"Min: 2016-01-01Max: 2021-12-30Duration: 2,190 days",589 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABNCAYAAACxBha+AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACkUlEQVR4nO3dTYriQByG8X+NrUSFiAi68wIDs3DpIeawc4RZeAJv4E4QiZjpYIyZzcAsuv1ItXnNx/PblwT6KVNVaYnL89wAhW+vvgC0B7FB5q2MD3XO9c2s5zH0lOf5+7OvB9Xw9Nicc/3JZPIzDMNx0bGHw2HvnPulCu4Lk8KMiVFYGd9svTAMx8vl8n00GiWPDoqiKFitVuPdbtczs9L/iF+ZFGb6idEEpdxGzcxGo1EynU7/FBzWL+ViPuc1Kcz0E6MpSoutLjwnhZl2YjQCu1HIEBtkiA0yxAaZRmwQPM/Lwsvl0i3jevC52sfme16WpmmQZdn3JEl+m5nPbhQF1T428zwv22w24/V6PTifz50yLw7/NSE2Myt+Xrbf71txTlal59SVii3Lsq6Zhc65IsNYe11RtefUN2NTLrzjOO6mafpjPp93Op3Ow7dD1l43Veo59dXY1Avv0+n0FgTBYLFYJLPZbP/oONZe91XlOfWtb7aXLLyHwyFrr4a6u2Zj4Y1n4QkCZIgNMsQGmUqds9WJ55mgWYt/u0BsHnzPBM3a/dsFYvPgeybY9t8uENsXFD0T/Ke1R0NsECDDN5tYmzcWxCbU9o0FsQm1fWNBbC9Qh41FGbd7YsMHZd3uiQ0flHW7JzZc9ezbPedskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkiA0yxAYZYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhl+XVUTTXghCbHVQFNeSEJsNdCUF5IQW43U/YUkbBAgQ2yQITbIEBtkiA0yxAYZYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhlig8zd/2eLoigo8oHH4zEwM4vjONhut4OmjavTtarH3WvlL6rovNJurq/hAAAAAElFTkSuQmCC"">",0(0.0%)
4,Año[int64],Mean (sd) : 2018.2 (1.7)min < med < max:2016.0 < 2018.0 < 2021.0IQR (CV) : 3.0 (1201.6),6 distinct values,"<img src = ""data:image/png;ba

In [69]:
homicidios_hechos.to_csv('Homicidios_hechos.csv') #nuevamente en un csv


Tratamiento de los datos del dataset homicidios- víctimas

In [36]:
homicidios_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  27605 non-null  object        
 6   SEXO              27605 non-null  object        
 7   EDAD_VICTIMA      27605 non-null  object        
 8   GRAVEDAD          27605 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 1.9+ MB


Transformación de los datos

In [37]:
print(homicidios_victimas.columns)

Index(['ID hecho', 'AAA', 'MM', 'DD', 'FECHA ', 'VEHICULO_VICTIMA', 'SEXO',
       'EDAD_VICTIMA', 'GRAVEDAD'],
      dtype='object')


In [50]:
# Se coloca la primera en mayúscula
homicidios_victimas.columns = [x.capitalize() for x in homicidios_victimas.columns]
# Se reemplazan los guiones por espacios
homicidios_victimas.columns = homicidios_victimas.columns.str.replace('_', ' ')
# Se renombran algunas columnas
homicidios_victimas = homicidios_victimas.rename(columns={'Id hecho': 'Id',
                                                      'FECHA ':'Fecha',   
                                                      'Aaa':'Año',
                                                      'Mm':'Mes',
                                                      'Dd':'Día',
                                                      'Victima':'Víctima'})
homicidios_victimas.columns

Index(['Id', 'Año', 'Mes', 'Día', 'Fecha ', 'Vehiculo victima', 'Sexo',
       'Edad victima', 'Gravedad'],
      dtype='object')

In [42]:
# Cambio la columna a tipo datetime
homicidios_victimas["Fecha "] = pd.to_datetime(homicidios_victimas["Fecha "], format='%Y-%m-%d')


In [43]:
#Una vez que se paso el tipo de dato a datetime que es una fecha y hora completa se utiliza este codigo para que solo se quede en formato fecha sin hora
homicidios_victimas["Fecha "] = homicidios_victimas["Fecha "].dt.date

#Luego se reemplazan las fechas en cero con espacios vacios a fin de su manipulación
homicidios_victimas["Fecha "] = homicidios_victimas ["Fecha "].replace(pd.NaT,"")

In [45]:
#Cambio la columna edad a numerico y luego a tipo entero
homicidios_victimas ['Edad victima'] = pd.to_numeric(homicidios_victimas['Edad victima'], errors="coerce")
homicidios_victimas ['Edad victima'] = homicidios_victimas['Edad victima'].astype("Int64")

In [46]:
#Utilizo Numpy para reemplazar por NaN los sin datos o vacios y luego paso a float para que cuando quiera aplicar estadisticas no tenga errores debido a esos datos
homicidios_victimas["Edad victima"] = homicidios_victimas["Edad victima"].replace([" ","SD"], np.nan)

In [47]:
homicidios_victimas["Edad victima"] = homicidios_victimas["Edad victima"].fillna(0.0)

In [48]:
#Controlo los datos de la columna
homicidios_victimas ["Edad victima"].unique()

<IntegerArray>
[ 57,   0,  45,  27,  32,  21,  37,  33,  25,  23,  42,  53,   1,  62,  67,
  75,  40,  48,   3,  49,  34,  52,  44,  28,  59,  50,  73,  26,  51,  16,
  43,  38,  64,  36,  84,  22,  20,   5,  39,  56,  69,  55,  46,  29,  41,
  54,  74,  70,   6,  30,  12,  83,  24,  35,  68,  15,  31,  19,   9,   2,
  13,  18,  17,  58,  60,  47,   7,  63,  66,  80,  65,  71,  79,  85,  61,
  77,  90,  76,  89,  78,  91,  97,  10,  72,  88,  11,  81,   8,  14,  96,
  82,  87,  86,   4,  92,  94, 100,  93,  95]
Length: 99, dtype: Int64

In [51]:
homicidios_victimas.head(3)

,Id,Año,Mes,Día,Fecha,Vehiculo victima,Sexo,Edad victima,Gravedad
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,0,SD
2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,0,SD


In [52]:
# se obeserva la cantidad de valores nulos en cada columna del df
homicidios_victimas.isnull().sum()

Id                  0
Año                 0
Mes                 0
Día                 0
Fecha               0
Vehiculo victima    0
Sexo                0
Edad victima        0
Gravedad            0
dtype: int64

Se crea la columna Rango etario basada en la columna Edad, para mejorar el análisis de los datos

In [53]:
# Se utiliza la función `pd.cut()`, que permite agrupar valores numéricos en intervalos o categorías


# Definir los límites de los rangos etarios y las etiquetas
bins = [0, 20, 40, 60, 80, 100]
labels = ['Menos de 20', ' Entre 21- 40', 'Entre 41-60', 'Entre 61-80', 'Más de 80']

# Utilizar pd.cut() para crear la columna de rango etario
homicidios_victimas['Rango_etario'] = pd.cut(homicidios_victimas['Edad victima'], bins=bins, labels=labels, right=False)

homicidios_victimas.head(3)

,Id,Año,Mes,Día,Fecha,Vehiculo victima,Sexo,Edad victima,Gravedad,Rango_etario
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD,Entre 41-60
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,0,SD,Menos de 20
2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,0,SD,Menos de 20


A continuación se estandarizan las mayúsculas y minúsculas del dataset

In [54]:
homicidios_hechos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     682 non-null    object        
 1   Num víctimas           682 non-null    int64         
 2   Fecha                  682 non-null    datetime64[ns]
 3   Año                    682 non-null    int64         
 4   Mes                    682 non-null    int64         
 5   Día                    682 non-null    int64         
 6   Hora completa          682 non-null    object        
 7   Hora                   681 non-null    Int64         
 8   Lugar del hecho        682 non-null    object        
 9   Tipo de calle          682 non-null    object        
 10  Calle                  682 non-null    object        
 11  Cruce                  525 non-null    object        
 12  Dirección normalizada  682 non-null    object        
 13  Comun

In [55]:
homicidios_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Id                27605 non-null  object  
 1   Año               27605 non-null  int64   
 2   Mes               27605 non-null  int64   
 3   Día               27605 non-null  int64   
 4   Fecha             27605 non-null  object  
 5   Vehiculo victima  27605 non-null  object  
 6   Sexo              27605 non-null  object  
 7   Edad victima      27605 non-null  Int64   
 8   Gravedad          27605 non-null  object  
 9   Rango_etario      27604 non-null  category
dtypes: Int64(1), category(1), int64(3), object(5)
memory usage: 1.9+ MB


A partir de los datos, se analiza eliminar columnas que se repiten en ambos dataset para realizar luego la unión y trabajar con un conjunto de datos. Se proceden a eliminar del dataset homicidios_victimas las columnas Fecha, Año, Mes, Día y Victima que contienen la misma información que se encuentra en homicidios_hechos.

Se realiza un resumen descriptivo de los datos en homicidios-victimas

In [71]:
homicidios_victimas.columns

Index(['Id', 'Año', 'Mes', 'Día', 'Fecha ', 'Vehiculo victima', 'Sexo',
       'Edad victima', 'Gravedad', 'Rango_etario'],
      dtype='object')

In [72]:
homicidios_victimas.Sexo

0        Varon
1           SD
2        Varon
3        Varon
4        Mujer
         ...  
27600    Varon
27601    Mujer
27602    Mujer
27603    Varon
27604    Varon
Name: Sexo, Length: 27605, dtype: object

In [64]:
dfSummary(homicidios_victimas)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Id[object],1. LC-2019-01884762. LC-2019-02910693. LC-2019-06553334. LC-2019-04281205. LC-2021-01376406. LC-2019-00498007. LC-2020-01121978. LC-2019-03484479. LC-2019-045947610. LC-2021-053341811. other,"16 (0.1%)16 (0.1%)10 (0.0%)10 (0.0%)10 (0.0%)10 (0.0%)8 (0.0%)8 (0.0%)8 (0.0%)8 (0.0%)27,501 (99.6%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADQklEQVR4nO3cwWkbYRRG0TdCJPZGQghUQtKDisg2HauINGGkReJ44yyCS8gdmJxTwb+48BDMp+X9/X2gsFv7Afw/xEZGbGTERkZsZMRGRmxkxEZGbGTERma3LMvXZVme1n4I27ebmS8z83nth7B9zigZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkdnNzI+Z+b32Q9i+xb8YUXFGyYiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyIiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNjXUXGuoqMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXghY/BCxhklIzYyYiNjXUXGuoqMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsa6iox1FRlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXghY/BCxhkls8zM88x8WvshbN7b/nw+fzscDqe1X8K2PR6Pl/3hcDhdr9dfx+Pxde0HsU33+/3pdrud9jMzx+Px9XK5/Fz7UWzasx8IZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZPYzfz8BWfshbNdHX8v5fP7u40n+tcfj8eKzcCpv1lVk/EAgIzYyYiMjNjJiIyM2MmIjIzYyYiPzB0WpV2yec8bJAAAAAElFTkSuQmCC"">",0(0.0%)
2,Año[int64],Mean (sd) : 2019.9 (0.9)min < med < max:2019.0 < 2020.0 < 2021.0IQR (CV) : 2.0 (2346.3),3 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABlklEQVR4nO3cQYriQBiG4apBRDcJIngYDzGHnUN4G2nKRdsMaGY3m4aOxoSZj36edVJEeYmU/FQdhqHA/+7Hv34AeMRq7IJa67aUsp64/u9hGK4T74W/vgy11rrd7/c/u67bTVn8crm81Vp/iZVXjb1R113X7Y7H47Xv+49nFm6tbU6n0+58Pq9LKULlJaM//aWU0vf9x+FweJ+w/nbCPfCJzRQRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEeGgoBR6x5OyyUJnF0rPLQmUui84uC5VZLTW7bDNFBKESQahEECoRhEoEoRJBqEQQKhH84b8A53XNT6gzc17XMoQ6P+d1LUCoC3Fe17xspoggVCIIlQhCJYJQiSBUIgiVCEIlglCJIFQiCJUIQiWCUIkgVCIIlQhCJYJQiSBUIgiVCEIlglCJIFQiCJUIQiWCUIkgVCIIlQhCJYJQiSBUIgiVCEIlglCJIFQiCJUID53h31rbPLtwa21zu91WpZSu1vr0gwXr7vf76ht+Zy997rFr/gAJKXcCZi4RzAAAAABJRU5ErkJggg=="">",0(0.0%)
3,Mes[int64],Mean (sd) : 6.6 (3.5)min < med < max:1.0 < 7.0 < 12.0IQR (CV) : 7.0 (1.9),12 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAC40lEQVR4nO3cQU8aQRyG8f8UMaOku0EJ3vgCJj2YePHSW4/9sL32VhNNevYbeDEmBJdAS1iBnmjapCo7w9R58fldycAenmV2Rhm3Wq0MyN27174AYBN7r30B2B3OuQMz2w8cPl+tVj+fepFQsRXOuYPj4+PPRVF0Q8aPx+ORc+7LU7G+GGrKuwQ7Zb8oiu7FxcXPsixnTQZWVeWvrq66w+Fw38yah5r6LsHuKcty1u/3fwQMPXjuxZe+UZPeJcCmNnpGTXWXAJtiewoSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSOCkFv0UeNlIsl8v2Nq/nT4QKM4s/bKSua79YLE5ns9k3Mwv5af2zCBVrwYeNmJnd3t52b25uDh8fH1spLo5Q8ZfQw0ZGo1HSw0aShrpYLNpmVjjnQt+CQ9ZgZglDnU6n7bquPwwGg1ar1Wo8lZi9zUPWIhc0Zjt6cycLdT6f73nvD8/OzmYnJyejpuOVD1mLiM0fHR19KsuyE/rZDw8PE+fcVzNr+uWQdNUeK/kzaqfTCT1gzUzwkLWY1XNd197MTs/Pz7/3er1x0/F3d3fvr6+vPw4Gg17TWSz1qj0Wi6ntC149r1fO3vs6dEETOoulXrXHItQnREzfxXK5bJdlOWoa27ZWziGzWOpVeyxC/YfY6TvnKVRV1qFGbm/FrH6jp+9cp1BV2YYau721ja2tkM3v3KdQVdmGGrO9VVWVv7y87A+Hw55zrvHq2TLfqnmLsg11LWRhEPttzHNmfrIPNUTsHxt4zszPToa6FvrHBp4z88N/+EMCoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoULCRr/rr6rKN33jyWTizcym06m/v78//J/jVT9b9bpjx2/S1y99u5HONrwMUAAAAABJRU5ErkJggg=="">",0(0.0%)
4,Día[int64],Mean (sd) :

In [70]:
homicidios_victimas.to_csv('Homicidios_victimas.csv') #nuevamente en un csv

Unificación de los dataset
Por medio de la función merge se unifican los dataset transformados anteriormente

In [65]:
siniestros= homicidios_victimas.merge(homicidios_hechos, on="Id", how="left")
siniestros.columns

Index(['Id', 'Año_x', 'Mes_x', 'Día_x', 'Fecha ', 'Vehiculo victima', 'Sexo',
       'Edad victima', 'Gravedad', 'Rango_etario', 'Num víctimas', 'Fecha',
       'Año_y', 'Mes_y', 'Día_y', 'Hora completa', 'Hora', 'Lugar del hecho',
       'Tipo de calle', 'Calle', 'Cruce', 'Dirección normalizada', 'Comuna',
       'Xy (caba)', 'Pos x', 'Pos y', 'Participantes', 'Victima', 'Acusado',
       'Coordenada x', 'Coordenada y', 'ID', 'OBJETO', 'Barrios', 'PERIMETRO',
       'AREA', 'Día semana'],
      dtype='object')

In [68]:
siniestros.Participantes

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
27600    NaN
27601    NaN
27602    NaN
27603    NaN
27604    NaN
Name: Participantes, Length: 27605, dtype: object

In [67]:
siniestros.dtypes

Id                               object
Año_x                             int64
Mes_x                             int64
Día_x                             int64
Fecha                            object
Vehiculo victima                 object
Sexo                             object
Edad victima                      Int64
Gravedad                         object
Rango_etario                   category
Num víctimas                    float64
Fecha                    datetime64[ns]
Año_y                           float64
Mes_y                           float64
Día_y                           float64
Hora completa                    object
Hora                              Int64
Lugar del hecho                  object
Tipo de calle                    object
Calle                            object
Cruce                            object
Dirección normalizada            object
Comuna                           object
Xy (caba)                        object
Pos x                           float64


In [66]:
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     27605 non-null  object        
 1   Año_x                  27605 non-null  int64         
 2   Mes_x                  27605 non-null  int64         
 3   Día_x                  27605 non-null  int64         
 4   Fecha                  27605 non-null  object        
 5   Vehiculo victima       27605 non-null  object        
 6   Sexo                   27605 non-null  object        
 7   Edad victima           27605 non-null  Int64         
 8   Gravedad               27605 non-null  object        
 9   Rango_etario           27604 non-null  category      
 10  Num víctimas           0 non-null      float64       
 11  Fecha                  0 non-null      datetime64[ns]
 12  Año_y                  0 non-null      float64       
 13  M

Al unificar los dataset se encontraron datos cambiados a tipo float que son necesarios como enteros, por lo que se transforman

In [122]:
siniestros["Num víctimas"].unique()

array([nan])

In [123]:
siniestros["Año"].unique()

array([nan])

In [125]:
siniestros.head(3)

,Id,Aaa,Mes_x,Día_x,Fecha,Vehiculo victima,Sexo,Edad victima,Gravedad,Rango etario,...,Victima,Acusado,Coordenada x,Coordenada y,Id y,Objeto,Barrios,Perimetro,Area,Día semana
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD,Entre 41-60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,0,SD,Menos de 20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,0,SD,Menos de 20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se realiza un resumen descriptivo de los datos de siniestros

In [126]:
siniestros.describe()

,Aaa,Mes_x,Día_x,Edad victima,Num víctimas,Fecha,Año,Mes_y,Día_y,Hora,Pos x,Pos y,Coordenada x,Coordenada y,Id y,Perimetro,Area
count,27605.000000,27605.000000,27605.000000,27605.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,2019.897265,6.618113,15.620721,31.616809,NaN,NaT,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2019.000000,1.000000,1.000000,0.0,NaN,NaT,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2019.000000,3.000000,8.000000,22.0,NaN,NaT,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2020.000000,7.000000,16.000000,31.0,NaN,NaT,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2021.000000,10.000000,23.000000,43.0,NaN,NaT,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,2021.000000,12.000000,31.000000,100.0,NaN,NaT,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.860899,3.479295,8.684067,19.678659,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
siniestros.columns

Index(['Id', 'Aaa', 'Mes_x', 'Día_x', 'Fecha ', 'Vehiculo victima', 'Sexo',
       'Edad victima', 'Gravedad', 'Rango etario', 'Num víctimas', 'Fecha',
       'Año', 'Mes_y', 'Día_y', 'Hora completa', 'Hora', 'Lugar del hecho',
       'Tipo de calle', 'Calle', 'Cruce', 'Dirección normalizada', 'Comuna',
       'Xy (caba)', 'Pos x', 'Pos y', 'Participantes', 'Victima', 'Acusado',
       'Coordenada x', 'Coordenada y', 'Id y', 'Objeto', 'Barrios',
       'Perimetro', 'Area', 'Día semana'],
      dtype='object')

In [135]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['Fecha ', 'Año', 'Mes_y', 'Día_y', 'Hora completa']

# Eliminar las columnas especificadas
siniestros = siniestros.drop(columnas_a_eliminar, axis=1)

In [136]:
siniestros.describe()

,Aaa,Mes_x,Día_x,Edad victima,Num víctimas,Fecha,Hora,Pos x,Pos y,Coordenada x,Coordenada y,Id y,Perimetro,Area
count,27605.000000,27605.000000,27605.000000,27605.0,0.0,27605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,2019.897265,6.618113,15.620721,31.616809,NaN,1900-01-01 00:00:00,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2019.000000,1.000000,1.000000,0.0,NaN,1900-01-01 00:00:00,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2019.000000,3.000000,8.000000,22.0,NaN,1900-01-01 00:00:00,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2020.000000,7.000000,16.000000,31.0,NaN,1900-01-01 00:00:00,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2021.000000,10.000000,23.000000,43.0,NaN,1900-01-01 00:00:00,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,2021.000000,12.000000,31.000000,100.0,NaN,1900-01-01 00:00:00,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.860899,3.479295,8.684067,19.678659,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
import pandas as pd

# Reemplaza NaT con una fecha por defecto o cualquier otro manejo que desees
siniestros['Fecha'] = siniestros['Fecha'].fillna(pd.to_datetime('1900-01-01'))

# Luego, puedes aplicar el formateo
dfSummary(siniestros)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Id[object],1. LC-2019-01884762. LC-2019-02910693. LC-2019-06553334. LC-2019-04281205. LC-2021-01376406. LC-2019-00498007. LC-2020-01121978. LC-2019-03484479. LC-2019-045947610. LC-2021-053341811. other,"16 (0.1%)16 (0.1%)10 (0.0%)10 (0.0%)10 (0.0%)10 (0.0%)8 (0.0%)8 (0.0%)8 (0.0%)8 (0.0%)27,501 (99.6%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADQklEQVR4nO3cwWkbYRRG0TdCJPZGQghUQtKDisg2HauINGGkReJ44yyCS8gdmJxTwb+48BDMp+X9/X2gsFv7Afw/xEZGbGTERkZsZMRGRmxkxEZGbGTERma3LMvXZVme1n4I27ebmS8z83nth7B9zigZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkdnNzI+Z+b32Q9i+xb8YUXFGyYiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyIiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNjXUXGuoqMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXghY/BCxhklIzYyYiNjXUXGuoqMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsa6iox1FRlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXghY/BCxhkls8zM88x8WvshbN7b/nw+fzscDqe1X8K2PR6Pl/3hcDhdr9dfx+Pxde0HsU33+/3pdrud9jMzx+Px9XK5/Fz7UWzasx8IZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZPYzfz8BWfshbNdHX8v5fP7u40n+tcfj8eKzcCpv1lVk/EAgIzYyYiMjNjJiIyM2MmIjIzYyYiPzB0WpV2yec8bJAAAAAElFTkSuQmCC"">",0(0.0%)
2,Aaa[int64],Mean (sd) : 2019.9 (0.9)min < med < max:2019.0 < 2020.0 < 2021.0IQR (CV) : 2.0 (2346.3),3 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABlklEQVR4nO3cQYriQBiG4apBRDcJIngYDzGHnUN4G2nKRdsMaGY3m4aOxoSZj36edVJEeYmU/FQdhqHA/+7Hv34AeMRq7IJa67aUsp64/u9hGK4T74W/vgy11rrd7/c/u67bTVn8crm81Vp/iZVXjb1R113X7Y7H47Xv+49nFm6tbU6n0+58Pq9LKULlJaM//aWU0vf9x+FweJ+w/nbCPfCJzRQRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEeGgoBR6x5OyyUJnF0rPLQmUui84uC5VZLTW7bDNFBKESQahEECoRhEoEoRJBqEQQKhH84b8A53XNT6gzc17XMoQ6P+d1LUCoC3Fe17xspoggVCIIlQhCJYJQiSBUIgiVCEIlglCJIFQiCJUIQiWCUIkgVCIIlQhCJYJQiSBUIgiVCEIlglCJIFQiCJUIQiWCUIkgVCIIlQhCJYJQiSBUIgiVCEIlglCJIFQiCJUID53h31rbPLtwa21zu91WpZSu1vr0gwXr7vf76ht+Zy997rFr/gAJKXcCZi4RzAAAAABJRU5ErkJggg=="">",0(0.0%)
3,Mes_x[int64],Mean (sd) : 6.6 (3.5)min < med < max:1.0 < 7.0 < 12.0IQR (CV) : 7.0 (1.9),12 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAC40lEQVR4nO3cQU8aQRyG8f8UMaOku0EJ3vgCJj2YePHSW4/9sL32VhNNevYbeDEmBJdAS1iBnmjapCo7w9R58fldycAenmV2Rhm3Wq0MyN27174AYBN7r30B2B3OuQMz2w8cPl+tVj+fepFQsRXOuYPj4+PPRVF0Q8aPx+ORc+7LU7G+GGrKuwQ7Zb8oiu7FxcXPsixnTQZWVeWvrq66w+Fw38yah5r6LsHuKcty1u/3fwQMPXjuxZe+UZPeJcCmNnpGTXWXAJtiewoSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSOCkFv0UeNlIsl8v2Nq/nT4QKM4s/bKSua79YLE5ns9k3Mwv5af2zCBVrwYeNmJnd3t52b25uDh8fH1spLo5Q8ZfQw0ZGo1HSw0aShrpYLNpmVjjnQt+CQ9ZgZglDnU6n7bquPwwGg1ar1Wo8lZi9zUPWIhc0Zjt6cycLdT6f73nvD8/OzmYnJyejpuOVD1mLiM0fHR19KsuyE/rZDw8PE+fcVzNr+uWQdNUeK/kzaqfTCT1gzUzwkLWY1XNd197MTs/Pz7/3er1x0/F3d3fvr6+vPw4Gg17TWSz1qj0Wi6ntC149r1fO3vs6dEETOoulXrXHItQnREzfxXK5bJdlOWoa27ZWziGzWOpVeyxC/YfY6TvnKVRV1qFGbm/FrH6jp+9cp1BV2YYau721ja2tkM3v3KdQVdmGGrO9VVWVv7y87A+Hw55zrvHq2TLfqnmLsg11LWRhEPttzHNmfrIPNUTsHxt4zszPToa6FvrHBp4z88N/+EMCoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoUICoULCRr/rr6rKN33jyWTizcym06m/v78//J/jVT9b9bpjx2/S1y99u5HONrwMUAAAAABJRU5ErkJggg=="">",0(0.0%)
4,Día_x[int64],Mean (s

In [140]:
siniestros.columns

Index(['Id', 'Aaa', 'Mes_x', 'Día_x', 'Vehiculo victima', 'Sexo',
       'Edad victima', 'Gravedad', 'Rango etario', 'Num víctimas', 'Fecha',
       'Hora', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Coordenada x', 'Coordenada y',
       'Id y', 'Objeto', 'Barrios', 'Perimetro', 'Area', 'Día semana'],
      dtype='object')

In [141]:
#Creo una lista con el orden 
nuevo_orden=["Id", "Fecha", "Aaa", "Mes_x", "Día_x","Día semana", "Hora", "Num víctimas", "Participantes", "Acusado", "Victima", "Sexo", "Edad victima", "Rango etario", "Lugar del hecho", "Tipo de calle", "Calle", "Cruce", "Dirección normalizada", "Comuna", "Barrios", "Xy (caba)", "Coordenada x", "Coordenada y", "Pos x", "Pos y"]

#Reorganizo el dataframe untilizando la lista y asignandola nuevamente aldataframe siniestros
siniestros_limpio = siniestros[nuevo_orden]
siniestros_limpio.columns

Index(['Id', 'Fecha', 'Aaa', 'Mes_x', 'Día_x', 'Día semana', 'Hora',
       'Num víctimas', 'Participantes', 'Acusado', 'Victima', 'Sexo',
       'Edad victima', 'Rango etario', 'Lugar del hecho', 'Tipo de calle',
       'Calle', 'Cruce', 'Dirección normalizada', 'Comuna', 'Barrios',
       'Xy (caba)', 'Coordenada x', 'Coordenada y', 'Pos x', 'Pos y'],
      dtype='object')

In [142]:
siniestros_limpio.head(3)

,Id,Fecha,Aaa,Mes_x,Día_x,Día semana,Hora,Num víctimas,Participantes,Acusado,...,Calle,Cruce,Dirección normalizada,Comuna,Barrios,Xy (caba),Coordenada x,Coordenada y,Pos x,Pos y
0,LC-2019-0000053,1900-01-01,2019,1,1,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LC-2019-0000063,1900-01-01,2019,1,1,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LC-2019-0000079,1900-01-01,2019,1,1,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
siniestros_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     27605 non-null  object        
 1   Fecha                  27605 non-null  datetime64[ns]
 2   Aaa                    27605 non-null  int64         
 3   Mes_x                  27605 non-null  int64         
 4   Día_x                  27605 non-null  int64         
 5   Día semana             0 non-null      object        
 6   Hora                   0 non-null      Int64         
 7   Num víctimas           0 non-null      float64       
 8   Participantes          0 non-null      object        
 9   Acusado                0 non-null      object        
 10  Victima                0 non-null      object        
 11  Sexo                   27605 non-null  object        
 12  Edad victima           27605 non-null  Int64         
 13  R

In [144]:
dfSummary(siniestros_limpio, is_collapsible=True)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Id[object],1. LC-2019-01884762. LC-2019-02910693. LC-2019-06553334. LC-2019-04281205. LC-2021-01376406. LC-2019-00498007. LC-2020-01121978. LC-2019-03484479. LC-2019-045947610. LC-2021-053341811. other,"16 (0.1%)16 (0.1%)10 (0.0%)10 (0.0%)10 (0.0%)10 (0.0%)8 (0.0%)8 (0.0%)8 (0.0%)8 (0.0%)27,501 (99.6%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADQklEQVR4nO3cwWkbYRRG0TdCJPZGQghUQtKDisg2HauINGGkReJ44yyCS8gdmJxTwb+48BDMp+X9/X2gsFv7Afw/xEZGbGTERkZsZMRGRmxkxEZGbGTERma3LMvXZVme1n4I27ebmS8z83nth7B9zigZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkdnNzI+Z+b32Q9i+xb8YUXFGyYiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyIiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNjXUXGuoqMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXghY/BCxhklIzYyYiNjXUXGuoqMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsa6iox1FRlnlIzYyBi8kDF4IeOMkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRMXghY/BCxhkls8zM88x8WvshbN7b/nw+fzscDqe1X8K2PR6Pl/3hcDhdr9dfx+Pxde0HsU33+/3pdrud9jMzx+Px9XK5/Fz7UWzasx8IZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZPYzfz8BWfshbNdHX8v5fP7u40n+tcfj8eKzcCpv1lVk/EAgIzYyYiMjNjJiIyM2MmIjIzYyYiPzB0WpV2yec8bJAAAAAElFTkSuQmCC"">",0(0.0%)
2,Fecha[datetime64[ns]],Min: 1900-01-01Max: 1900-01-01Duration: 0 days,1 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABNCAYAAACxBha+AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABeElEQVR4nO3XwWlCQRiF0ZkgRjfvIYLFWESKTRF2I2FcRBPQSQMhqMQr4jnb4Ye7+DZTe+8FEl7uPYDnITZiJvce8KhqrfNSyvSK0+/e+/6/9zwCsV2h1jpfLpdvwzAsLr3d7XYftdb3ZwxObNeZDsOwWK/X+3EcD+cetdZmm81msd1up6UUsXG+cRwPq9Xq88Kz+U3GPAAfBGLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjZvLXY611Vkp5DW15JMPpdJq01maXHLXWZsfjcVJKGWqtN5p2d1+998NvDz9uRjEePRreyAAAAABJRU5ErkJggg=="">",0(0.0%)
3,Aaa[int64],Mean (sd) : 2019.9 (0.9)min < med < max:2019.0 < 2020.0 < 2021.0IQR (CV) : 2.0 (2346.3),3 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABlklEQVR4nO3cQYriQBiG4apBRDcJIngYDzGHnUN4G2nKRdsMaGY3m4aOxoSZj36edVJEeYmU/FQdhqHA/+7Hv34AeMRq7IJa67aUsp64/u9hGK4T74W/vgy11rrd7/c/u67bTVn8crm81Vp/iZVXjb1R113X7Y7H47Xv+49nFm6tbU6n0+58Pq9LKULlJaM//aWU0vf9x+FweJ+w/nbCPfCJzRQRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEeGgoBR6x5OyyUJnF0rPLQmUui84uC5VZLTW7bDNFBKESQahEECoRhEoEoRJBqEQQKhH84b8A53XNT6gzc17XMoQ6P+d1LUCoC3Fe17xspoggVCIIlQhCJYJQiSBUIgiVCEIlglCJIFQiCJUIQiWCUIkgVCIIlQhCJYJQiSBUIgiVCEIlglCJIFQiCJUIQiWCUIkgVCIIlQhCJYJQiSBUIgiVCEIlglCJIFQiCJUID53h31rbPLtwa21zu91WpZSu1vr0gwXr7vf76ht+Zy997rFr/gAJKXcCZi4RzAAAAABJRU5ErkJggg=="">",0(0.0%)
4,Mes_x[int64],Mean (sd) : 6.6 (3.5)min < med < max:1.0 < 7.0 < 12.0IQR (CV) : 7.0 (1.9),12 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAC40lEQVR4nO3cQU8aQRyG8f8UMaOku0EJ3vgCJj2YePHSW4/9sL32VhNNevYbeDEmBJdAS1iBnmjapCo7w9R58fldycAenmV2Rhm3Wq0MyN27174AYBN7r30B2B3OuQMz2w8cPl+tVj+fepFQsRXOuYPj4+PPRVF0Q8aPx+ORc+7LU7G+GGrKuwQ7Zb8oiu7FxcXPsixnTQZWVeWvrq66

Se guarda el archivo como csv para optimizar su manipulación

In [146]:
import os

# Ruta del directorio
directorio = "data"

# Verificar si el directorio existe
if not os.path.exists(directorio):
    # Crear el directorio si no existe
    os.makedirs(directorio)

# Guardar el DataFrame en un archivo CSV en el directorio especificado
siniestros_limpio.to_csv(os.path.join(directorio, "siniestros_limpio.csv"), index=False)


In [147]:
siniestros_limpio.to_csv("data/siniestos_limpio.csv", index=False)